# Load and test trained model

## Load libraries

In [1]:
import numpy as np
import torch
import torch.nn as nn
from enduro_lstm import *

In [2]:
device = conf_cuda(False)

Selected CPU


In [3]:
device

device(type='cpu')

In [4]:
device.type

'cpu'

## Set configurations

In [5]:
data_path = r"../1-generate/data/"
match = 35
use_cuda = False
zigzag = False

## Load trained model

In [6]:
newpath = 'models/play_m45to49_f1to1000_epoch10000_H500/' + 'play_m45to49_f1to1000_epoch10000_H500'

In [7]:
hidden_neurons = 500
load_checkpoint = False

In [8]:
if zigzag:
    output_size = 2
else:
    output_size = 9

In [9]:
def load_checkpoint(model, filename='checkpoint.pth.tar'):
    
    print("=> loading checkpoint '{}'".format(filename))
    checkpoint = torch.load(filename, map_location=device)
    model.load_state_dict(checkpoint['state_dict'])

    return model

In [10]:
if load_checkpoint:
    model = Model(device=device, input_size=20400, output_size=output_size, hidden_dim=hidden_neurons, n_layers=1)
    model = load_checkpoint(model, newpath)
else:
    model = Model(device=device, input_size=20400, output_size=output_size, hidden_dim=hidden_neurons, n_layers=1)
    model.load_state_dict(torch.load(newpath, map_location=device))
model.eval()

=> loading checkpoint 'models/play_m45to49_f1to1000_epoch10000_H500/play_m45to49_f1to1000_epoch10000_H500'


Model(
  (lstm): LSTM(20400, 500, batch_first=True)
  (fc): Linear(in_features=500, out_features=9, bias=True)
  (sigmoid): Sigmoid()
)

In [60]:
model = Model(device=device, input_size=20400, output_size=output_size, hidden_dim=hidden_neurons, n_layers=1)
torch.load(newpath, map_location=device)
model.eval()

Model(
  (lstm): LSTM(20400, 200, batch_first=True)
  (fc): Linear(in_features=200, out_features=9, bias=True)
  (sigmoid): Sigmoid()
)

## Prepare cell with trained model 

In [11]:
lstmcell = nn.LSTMCell(20400, hidden_neurons)
linear = nn.Linear(hidden_neurons, output_size)
sigmoid = nn.Sigmoid()

In [12]:
lstmcell.weight_ih = model.lstm.weight_ih_l0
lstmcell.weight_hh = model.lstm.weight_hh_l0
lstmcell.bias_hh = model.lstm.bias_hh_l0
lstmcell.bias_ih = model.lstm.bias_ih_l0
linear.weight = model.fc.weight
linear.bias = model.fc.bias

In [13]:
hx = torch.zeros(1, hidden_neurons)
cx = torch.zeros(1, hidden_neurons)

In [14]:
print(model.lstm.weight_ih_l0.shape)
print(model.lstm.weight_hh_l0.shape)
print(model.lstm.bias_ih_l0.shape)
print(model.lstm.bias_hh_l0.shape)
print(model.fc.weight.shape)
print(model.fc.bias.shape)

torch.Size([2000, 20400])
torch.Size([2000, 500])
torch.Size([2000])
torch.Size([2000])
torch.Size([9, 500])
torch.Size([9])


## Testing outputs of model

In [264]:
ACTIONS_LIST = get_actions_list(zigzag=zigzag)

In [265]:
num_of_frames, frames, actions, rewards, lifes = load_npz(data_path, match)

Successfully loaded NPZ.


In [266]:
X_train = frames.reshape(1, len(frames) ,-1)/255

In [267]:
ACTIONS_LIST

[2, 3]

In [268]:
actions

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [269]:
action_one_hot = [prepare_action_data(i, ACTIONS_LIST) for i in actions]
Y_train = np.array(action_one_hot)
Y_train = Y_train.reshape(1, len(Y_train), -1)

In [270]:
X_train = torch.tensor(X_train).float()
Y_train = torch.tensor(Y_train).float()

In [271]:
hx = torch.zeros(1, hidden_neurons)
cx = torch.zeros(1, hidden_neurons)
output = []
for i in range(120):
    step_input = X_train[0][i]
    step_input = step_input.reshape(1, -1)
    hx, cx = lstmcell(step_input, (hx, cx))
    out = linear(hx)
    out = sigmoid(out)
    output.append(out)

In [272]:
from sklearn.metrics import confusion_matrix

In [273]:
Y_train = Y_train.reshape(-1, len(ACTIONS_LIST))

In [274]:
acertou = 0
errou = 0
for i in range(120):
    if torch.argmax(Y_train[i]) == torch.argmax(output[i]):
        acertou += 1
    else:
        errou += 1 

In [275]:
output

[tensor([[0.9799, 0.0189]], grad_fn=<SigmoidBackward>),
 tensor([[0.9834, 0.0158]], grad_fn=<SigmoidBackward>),
 tensor([[0.9870, 0.0124]], grad_fn=<SigmoidBackward>),
 tensor([[0.9848, 0.0145]], grad_fn=<SigmoidBackward>),
 tensor([[0.9871, 0.0123]], grad_fn=<SigmoidBackward>),
 tensor([[0.9873, 0.0121]], grad_fn=<SigmoidBackward>),
 tensor([[0.9863, 0.0131]], grad_fn=<SigmoidBackward>),
 tensor([[0.9879, 0.0115]], grad_fn=<SigmoidBackward>),
 tensor([[0.9891, 0.0104]], grad_fn=<SigmoidBackward>),
 tensor([[0.9890, 0.0105]], grad_fn=<SigmoidBackward>),
 tensor([[0.9882, 0.0112]], grad_fn=<SigmoidBackward>),
 tensor([[0.9881, 0.0113]], grad_fn=<SigmoidBackward>),
 tensor([[0.9882, 0.0112]], grad_fn=<SigmoidBackward>),
 tensor([[0.9887, 0.0108]], grad_fn=<SigmoidBackward>),
 tensor([[0.9879, 0.0115]], grad_fn=<SigmoidBackward>),
 tensor([[0.9888, 0.0107]], grad_fn=<SigmoidBackward>),
 tensor([[0.9872, 0.0122]], grad_fn=<SigmoidBackward>),
 tensor([[0.9882, 0.0112]], grad_fn=<SigmoidBack

In [281]:
acertou

120

In [282]:
errou

0

In [283]:
acertou/(acertou + errou)

1.0

## Play Gym Enduro

In [15]:
import gym
import time
from PIL import Image

In [16]:
if zigzag:
        
    ACTIONS = {
        "right": 2,
        "left": 3,
    }

else:

    ACTIONS = {
        "noop": 0,
        "accelerate": 1,
        "right": 2,
        "left": 3,
        "break": 4,
        "right_break": 5,
        "left_break": 6,
        "right_accelerate": 7,
        "left_accelerate": 8,
    }

In [17]:
y_min, y_max, x_min, x_max = 25, 195, 20, 140
shape_of_single_frame = (1, (y_max-y_min),(x_max-x_min))

In [18]:
sleep_time = 0.05

In [23]:
env = gym.make("Enduro-v0")
frame = env.reset()
reward, action, done, info = 0, 0, False, {'ale.lives': 0}

hx = torch.zeros(1, hidden_neurons)
cx = torch.zeros(1, hidden_neurons)

while(1000):
    
    time.sleep(sleep_time)
    env.render()
    
    frame = frame[y_min:y_max, x_min:x_max]

    frame = Image.fromarray(frame)
    frame = frame.convert("L")
    
    frame = np.asarray(frame)
    frame = frame.reshape(1, -1)
    frame = torch.tensor(frame)/255
    
    hx, cx = lstmcell(frame, (hx, cx))
    out = linear(hx)
    action = sigmoid(out)
    
    action = list(ACTIONS.values())[torch.argmax(action, axis=1)]
    print(action)
    frame, reward, done, info = env.step(action)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
3
3
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
1
1
1
1
1
1
1
1
3
1
1
1
1
1
1
1
1
3
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
3
3
3
1
1
1
1
1
1
1
1
3
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
3
3
3
0
0
0
0
0
0
0
0
0
0
0
3
3
3
3
3
2
3
2
3
3
3
0
0
0
0
0
0
0
0
0
0
3
1
1
1
1
1
1
1
1
8
8
8
8
8
8
8
8
1
1
1
1
1
3
3
8
1
1
1
3
3
1
1
1
1
1
1
3
1
1
1
1
1
1
0
0
0
3
0
0
0
0
0
3
3
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
0
0
1
1
0
1
1
1
1
1
1
1


KeyboardInterrupt: 